<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/Clase_Aprendizaje_Profundo/blob/main/L10_Poda_neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizaje Profundo
# Neural Pruning
##Dr. Carlos Villaseñor

(Ejemplo basado en [enlace](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras))



Paso 1. Instalamos la biblioteca para

In [ ]:
! pip install -q tensorflow-model-optimization

Paso 2. Importamos los paquetes necesarios

In [ ]:
# Bibliotecas principales
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Paquetería de poda neuronal
import tensorflow_model_optimization as tfmot

## Entrenar el modelo base

Paso 3. Importamos los datos de MNIST y normalizamos las imagenes

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

Paso 4. Creamos una red neuronal convolucional

In [ ]:
# Definimos la arquitectura
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Compilamos el modelo
model.compile(optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

model.summary()

Paso 5. Entrenamos el modelo

In [ ]:
model.fit(train_images, train_labels, epochs=4, validation_split=0.1)

Paso 6. Evaluamos el modelo

In [ ]:
_, base_model_acc = model.evaluate(test_images, test_labels, verbose=0)
print('Original model :', base_model_acc)

Original model : 0.9753999710083008


Paso 7. Guardamos el modelo en memoria secundaria

In [ ]:
tf.keras.models.save_model(model, 'base_model.h5', include_optimizer=False)

## Poda neuronal

Paso 8. Empecemos a usar la poda neuronal, definamos un modelo especial para hacer la poda. En el siguiente bloque prepararemos un modelo para hacer prunning sobre todo el modelo, también es posible hacer prunning en capas especificas como pueden verlo en este [enlace](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/prune_low_magnitude).

In [ ]:
# Creamos objeto de poda para quitar
Pruner = tfmot.sparsity.keras.prune_low_magnitude


# Necesitamos calcular cuantos pasos daremos en el proceso de poda
# para esto definimos los siguientes parámetros
batch_size = 128
epochs = 2
validation_split = 0.1

# Calcular end-step
num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Definimos los parámetros para el esquema de poda
pruning_params = {
'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                           final_sparsity=0.80,
                                                           begin_step=0,
                                                           end_step=end_step)
}

# Creamos el modelo para podar
pruned_model = Pruner(model, **pruning_params)

# Es necesario recompilar el modelo.
pruned_model.compile(optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

pruned_model.summary()

Paso 9. El proceso de podado neuronal. Como cualquier otro modelo entrenemos el modelo original con un proceso de poda.

In [ ]:
# Creamos un par de callback para usar la poda
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

# Pdamos nuestro modelo
pruned_model.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs,
                  validation_split=validation_split,
                  callbacks=callbacks)

Paso 10. Comparemos ambos modelos.

In [ ]:
_, pruned_model_acc = pruned_model.evaluate(test_images, test_labels, verbose=0)

print('Original test accuracy:', base_model_acc)
print('Pruned test accuracy:', pruned_model_acc)

Paso 11. Comprimimos el modelo. Ahora que el modelo ha sido padado. Muchos de las neuronas no tienen contribución pero el modelo. es igual de grande; ahora vamos a comprimirlo para tener un modelo más pequeño en memoria.

In [ ]:
compressed_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

print(compressed_model.summary())

tf.keras.models.save_model(compressed_model, 'compressed_model.h5',
                           include_optimizer=False)

## TensorFlow Lite

Paso 12. Para hadware de bajo rendimiento es mejor usar una versión reducida de TensorFlow, llamada TensorFlow Lite. Este framework reducido permite correr modelos de Aprendizaje profundo en hadwares pequeños o embebidos.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(compressed_model)
compressed_model_lite = converter.convert()

with open('compressed_model_lite.tflite', 'wb') as f:
  f.write(compressed_model_lite)

# Poda y cuantización

Paso 13. Usaremos la poda neuronal en conjunto con las técnica de cuantización para comprimir nuestro modelo aun más.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(compressed_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_lite_model = converter.convert()

with open('quantized_and_pruned_model.tflite', 'wb') as f:
  f.write(quantized_and_pruned_lite_model)

Paso 14. Para poder comparar el modelo anterior, sirvase de la siguiente función que evaluara el modelo de TFLite interpretado dentro de nuestro entorno.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy


interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_lite_model)
interpreter.allocate_tensors()
test_accuracy = evaluate_model(interpreter)

## Comparación final

Paso 15. Finalmente vemos la comparación entre

In [ ]:
print('Modelo original:', base_model_acc)
print('Modelo podado:', pruned_model_acc)
print('Modelo podado y cuantizado para TFlite:', test_accuracy)